#### Page1: Importing Libraries and Connections
######**------------------------  ONLY EXECUTE  ------------------------ **

In [0]:
#--------- LIBRARIES ---------#
from delta.tables import *
import pyspark
from pyspark.sql.types import * 
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from datetime import datetime


#--------- SNOWFLAKE CONNECTIONS ---------#
sfoptions = {
  "sfUrl": "https://nonprodthreemobile.west-europe.azure.snowflakecomputing.com/",
  "sfUser": "SVC_SIT_ALDM",
  "sfPassword": "******",
  "sfDatabase": "SIT_ALDM",
  "sfSchema": "ALDM",
  "sfWarehouse": "SIT_ALDM_WH"
}

#### Page2: Input Source and Target Queries -
###### Note: Query should be in a single line, no need to add semicolon at the end of the query

In [0]:
##----------------------------------------------##
#          SOURCE QUERY - DATABRICKS
##----------------------------------------------##

Source_Query="select A.* from ( select distinct trim(table_act_entry.OBJID) as ACTIVITY_KEY, trim(TABLE_ACT_ENTRY.ACT_CODE) as ACTIVITY_CODE,trim(entry_time) as ACTIVITY_DATE, trim(addnl_info) as ACTIVITY_DESCRIPTION, trim(substr(table_phone_log.notes,1,400)) as ACTIVITY_LONG_DESCRIPTION, trim(TABLE_ACT_ENTRY.entry_name2gbst_elm) as ACTIVITY_TYPE_KEY, trim(table_blg_argmnt.BAR_ID) as BILLING_ARRANGEMENT_KEY, coalesce(TABLE_ACT_ENTRY.ACT_ENTRY2CONTACT,TABLE_CONTACT.OBJID)  as CONTACT_KEY, trim(COALESCE(table_customer.CUSTOMER_ID, TABLE_FIN_ACCNT.FA_ID)) as CUSTOMER_KEY, trim(TABLE_ACT_ENTRY.act_entry2user) as EMPLOYEE_KEY, trim(table_fin_accnt.fa_id) as FINANCIAL_ACCOUNT_KEY, case when table_act_entry.OBJID <0 then 'Y' else NULL END as MIGRATION_IND, trim(TABLE_ACT_ENTRY.ACT_ENTRY2CASE) as SERVICE_REQUEST_CASE_KEY, date_format(cast(TABLE_ACT_ENTRY.REPL_TIMESTAMP as TIMESTAMP),'y-MM-dd') as GS_INSERT_TS from ALDM_STAGING.TABLE_ACT_ENTRY left join ALDM_STAGING.table_blg_argmnt on table_blg_argmnt.objid=TABLE_ACT_ENTRY.focus_lowid and TABLE_ACT_ENTRY.focus_type=5695 left join ALDM_STAGING.table_customer on table_customer.objid=TABLE_ACT_ENTRY.focus_lowid and TABLE_ACT_ENTRY.focus_type=6490 left join ALDM_STAGING.table_phone_log on table_act_entry.act_entry2phone_log=table_phone_log.objid left join (SELECT ELM.OBJID as CRM_ACTIVITY_TYPE_KEY FROM ALDM_STAGING.table_gbst_elm ELM, ALDM_STAGING.table_gbst_lst LST WHERE elm.GBST_ELM2GBST_LST = lst.objid and lst.title = 'Activity Name') CRM_ACTIVITY_TYPE on TABLE_ACT_ENTRY.entry_name2gbst_elm=CRM_ACTIVITY_TYPE.CRM_ACTIVITY_TYPE_KEY left join ALDM_STAGING.table_contact on trim(TABLE_ACT_ENTRY.act_entry2contact)=trim(table_contact.OBJID) left join ALDM_STAGING.TABLE_USER as Employee on TABLE_ACT_ENTRY.act_entry2user = Employee.OBJID left JOIN ALDM_STAGING.table_fin_accnt on table_fin_accnt.objid=TABLE_ACT_ENTRY.focus_lowid and TABLE_ACT_ENTRY.focus_type=5694 left join ALDM_STAGING.TABLE_CASE as SERVICE_REQUEST_CASE on TABLE_ACT_ENTRY.ACT_ENTRY2CASE=SERVICE_REQUEST_CASE.OBJID) A where A.GS_INSERT_TS<'2022-09-06'"

##----------------------------------------------##
#           TARGET QUERY - SNOWFLAKE
##----------------------------------------------##

TableName='CRM_ACTIVITY'
targetQuery = "select ACTIVITY_KEY,ACTIVITY_CODE ,to_char(ACTIVITY_DATE,'YYYY-MM-DD HH24:MI:SS') as ACTIVITY_DATE ,ACTIVITY_DESCRIPTION ,ACTIVITY_LONG_DESCRIPTION ,ACTIVITY_TYPE_KEY ,BILLING_ARRANGEMENT_KEY ,CONTACT_KEY ,CUSTOMER_KEY ,EMPLOYEE_KEY ,FINANCIAL_ACCOUNT_KEY ,MIGRATION_IND ,SERVICE_REQUEST_CASE_KEY,to_char(GG_INSERT_TS,'YYYY-MM-DD') as GG_INSERT_TS from CRM_ACTIVITY where to_char(GG_INSERT_TS,'YYYY-MM-DD')<'2022-09-06'"

##----------------------------------------------##
#                   AUDIT_REF
##----------------------------------------------##

TABLE_ID=spark.sql("select coalesce(max(TABLE_ID),0)+1 as Query_id from AUDIT_REF").collect()[0][0]
print("TABLE_ID:",TABLE_ID)
Source_Type='Databricks'
Target_type='Snowflake'
Source_Query=Source_Query.replace("'", "\\'")
Target_Query=targetQuery.replace("'", "\\'")
Insert_query = "insert into AUDIT_REF values("+ str(TABLE_ID) + ",'" + TableName + "','" + Source_Type + "','" + Target_type + "','" + Source_Query + "','" + Target_Query + "',CURRENT_TIMESTAMP)" 
spark.sql(Insert_query)
spark.sql("SELECT * FROM AUDIT_REF WHERE TABLE_ID=" + str(TABLE_ID) ).display()


TABLE_ID: 1

TABLE_ID,TABLE_NAME,SOURCE_TYPE,TARGET_TYPE,SRC_QUERY,TGT_QUERY,INSERT_TIME
1,CRM_ACTIVITY,Databricks,Snowflake,"select A.* from ( select distinct trim(table_act_entry.OBJID) as ACTIVITY_KEY, trim(TABLE_ACT_ENTRY.ACT_CODE) as ACTIVITY_CODE,trim(entry_time) as ACTIVITY_DATE, trim(addnl_info) as ACTIVITY_DESCRIPTION, trim(substr(table_phone_log.notes,1,400)) as ACTIVITY_LONG_DESCRIPTION, trim(TABLE_ACT_ENTRY.entry_name2gbst_elm) as ACTIVITY_TYPE_KEY, trim(table_blg_argmnt.BAR_ID) as BILLING_ARRANGEMENT_KEY, coalesce(TABLE_ACT_ENTRY.ACT_ENTRY2CONTACT,TABLE_CONTACT.OBJID) as CONTACT_KEY, trim(COALESCE(table_customer.CUSTOMER_ID, TABLE_FIN_ACCNT.FA_ID)) as CUSTOMER_KEY, trim(TABLE_ACT_ENTRY.act_entry2user) as EMPLOYEE_KEY, trim(table_fin_accnt.fa_id) as FINANCIAL_ACCOUNT_KEY, case when table_act_entry.OBJID <0 then 'Y' else NULL END as MIGRATION_IND, trim(TABLE_ACT_ENTRY.ACT_ENTRY2CASE) as SERVICE_REQUEST_CASE_KEY, date_format(cast(TABLE_ACT_ENTRY.REPL_TIMESTAMP as TIMESTAMP),'y-MM-dd') as GS_INSERT_TS from ALDM_STAGING.TABLE_ACT_ENTRY left join ALDM_STAGING.table_blg_argmnt on table_blg_argmnt.objid=TABLE_ACT_ENTRY.focus_lowid and TABLE_ACT_ENTRY.focus_type=5695 left join ALDM_STAGING.table_customer on table_customer.objid=TABLE_ACT_ENTRY.focus_lowid and TABLE_ACT_ENTRY.focus_type=6490 left join ALDM_STAGING.table_phone_log on table_act_entry.act_entry2phone_log=table_phone_log.objid left join (SELECT ELM.OBJID as CRM_ACTIVITY_TYPE_KEY FROM ALDM_STAGING.table_gbst_elm ELM, ALDM_STAGING.table_gbst_lst LST WHERE elm.GBST_ELM2GBST_LST = lst.objid and lst.title = 'Activity Name') CRM_ACTIVITY_TYPE on TABLE_ACT_ENTRY.entry_name2gbst_elm=CRM_ACTIVITY_TYPE.CRM_ACTIVITY_TYPE_KEY left join ALDM_STAGING.table_contact on trim(TABLE_ACT_ENTRY.act_entry2contact)=trim(table_contact.OBJID) left join ALDM_STAGING.TABLE_USER as Employee on TABLE_ACT_ENTRY.act_entry2user = Employee.OBJID left JOIN ALDM_STAGING.table_fin_accnt on table_fin_accnt.objid=TABLE_ACT_ENTRY.focus_lowid and TABLE_ACT_ENTRY.focus_type=5694 left join ALDM_STAGING.TABLE_CASE as SERVICE_REQUEST_CASE on TABLE_ACT_ENTRY.ACT_ENTRY2CASE=SERVICE_REQUEST_CASE.OBJID) A where A.GS_INSERT_TS<'2022-09-06'","select ACTIVITY_KEY,ACTIVITY_CODE ,to_char(ACTIVITY_DATE,'YYYY-MM-DD HH24:MI:SS') as ACTIVITY_DATE ,ACTIVITY_DESCRIPTION ,ACTIVITY_LONG_DESCRIPTION ,ACTIVITY_TYPE_KEY ,BILLING_ARRANGEMENT_KEY ,CONTACT_KEY ,CUSTOMER_KEY ,EMPLOYEE_KEY ,FINANCIAL_ACCOUNT_KEY ,MIGRATION_IND ,SERVICE_REQUEST_CASE_KEY,to_char(GG_INSERT_TS,'YYYY-MM-DD') as GG_INSERT_TS from CRM_ACTIVITY where to_char(GG_INSERT_TS,'YYYY-MM-DD')<'2022-09-06'",2022-09-15 18:04:42.005


#### Data Comparison ( Execute Only )

In [0]:
###------------------------  ONLY EXECUTE  ------------------------ ###

now = datetime.now()
START_TIME = now.strftime("%Y-%m-%d %H:%M:%S")
## Collecting data from Source and Target into DataFrames 

##----------------SOURCE DATAFRAME------------------------------##
Source_Query=spark.sql("SELECT * FROM AUDIT_REF WHERE TABLE_ID=" + str(TABLE_ID) ).collect()[0].SRC_QUERY.replace("\\'", "'")
DF_SRC=spark.sql(Source_Query)
#print("Source Record Count: ",DF_SRC.count())

##----------------TARGET DATAFRAME-----------------------------##
Target_Query=spark.sql("SELECT * FROM AUDIT_REF WHERE TABLE_ID=" + str(TABLE_ID) ).collect()[0].TGT_QUERY.replace("\\'", "'")
Load_targetQuery = "(" + Target_Query + ") foo"
DF_TGT=spark.read \
.format("net.snowflake.spark.snowflake") \
.options(**sfoptions) \
.option("query", Load_targetQuery) \
.load()
#print("Target Record Count: ",DF_TGT.count())

# Data Comparison
from pyspark.sql.functions import md5,concat_ws

#Defining all the variables
source_count =0
target_count=0
source_no_of_columns=0
target_no_of_columns=0
target_column_names_str={}
source_column_names_str={}
status=0



#SchemaCompare Function without try-catch block
def schemaCompare(DF_SRC,DF_TGT):
    try:
        #Counting the number of Columns
        source_no_of_columns=len(DF_SRC.columns)
        target_no_of_columns=len(DF_TGT.columns)
        #Getting the name of Columns as list and then converting it to string to print
        source_column_names=DF_SRC.columns
        target_column_names=DF_TGT.columns
        source_column_names_str=','.join([str(item) for item in source_column_names])
        target_column_names_str=','.join([str(item) for item in target_column_names])
        #Print the names of Source and target Column 
        print("Source Column are : "+source_column_names_str)
        print("Target Columns are : "+target_column_names_str)
        if(source_no_of_columns==target_no_of_columns):
            print("Source Column Count : " + str(source_no_of_columns))
            print("Target Column Count : " + str(target_no_of_columns))
            print("Column Count validation is successful")
            print("---------------------------------------------------------")
        else:
            print("Schema validation failed | Source Column count: " + str(source_no_of_columns) + " Target Column Count: " + str(target_no_of_columns) + " | Process will now terminate !!")
            print("---------------------------------------------------------")
            status=1
    except Exception as e:
        print("Failed With Exception : " + str(e))
        status=1
        
    
        
        
schemaCompare(DF_SRC,DF_TGT)


#CountCompare Function without try-catch block
def countCompare(DF_SRC,DF_TGT):
    try:
        #Counting the number of Rows
        source_count=DF_SRC.count()
        target_count=DF_TGT.count()
        if(source_count==target_count):
            print("Source Record Count : " + str(source_count))
            print("Target Record Count : " + str(target_count))
            print("\n")
            print("Count validation is successful")
            print("---------------------------------------------------------")
        else:
            print("Count validation failed | Source count: " + str(source_count) + " Target Count: " + str(target_count) + " | Process will now terminate !!")
            print("---------------------------------------------------------")
            status=1
    except Exception as e:
        print("Failed With Exception : " + str(e))
        status=1
        
    
        
        
countCompare(DF_SRC,DF_TGT)


#Comparing Data of Both dataframes
def dataCompare(DF_SRC,DF_TGT):
    try:
        
        #print("Source Preview one sample record: ")
        #DF_SRC.display(1)

        #print("Target Preview one sample record: ")
        #DF_TGT.display(1)


        #Performing MD5 Operation
        DF_SRC=DF_SRC.withColumn("Md5Hash",md5(concat_ws("~",*DF_SRC.columns)))
        DF_TGT=DF_TGT.withColumn("Md5Hash",md5(concat_ws("~",*DF_TGT.columns)))
        spark.sql("DROP VIEW IF EXISTS df_src_hashed")
        spark.sql("DROP VIEW IF EXISTS df_tgt_hashed")
        DF_SRC.createTempView("df_src_hashed")
        DF_TGT.createTempView("df_tgt_hashed")

        #Query to find difference in 
        md5_diff=spark.sql("select COALESCE(A.Md5Hash,B.Md5Hash) as Md5Hash from (select Md5Hash,count(1) as cn_cnt from df_src_hashed group by Md5Hash) A full outer join (select Md5Hash,count(1) as cn_cnt from df_tgt_hashed group by Md5Hash) B ON A.cn_cnt=B.cn_cnt AND A.Md5Hash=B.Md5Hash WHERE (B.Md5Hash is null OR A.Md5Hash is null) group by 1")
        spark.sql("DROP VIEW IF EXISTS md5_differences")
        md5_diff.createTempView("md5_differences")
        diff_cnt=md5_diff.count()/2
        
        if (diff_cnt == 0):
            #Data is Completely same
            print("Data is completely matching between source and target")
            return "Data is completely matching between source and target: " + str(diff_cnt)
        else:
            #Data has differences
            print("Data discrepancies are identified between the Source and Target. Total count of discrepancies is : "+str(int(diff_cnt)))

            #Getting the name of Columns as list and then converting it to string to print
            source_column_names=DF_SRC.columns
            source_column_names.remove('Md5Hash')
            print(source_column_names)
            source_column_names_str=','.join([str(item) for item in source_column_names])
            target_column_names=DF_TGT.columns
            target_column_names.remove('Md5Hash')
            print(target_column_names)
            target_column_names_str=','.join([str(item) for item in target_column_names])


            #Dataframe to store differences
            md5_values_target = spark.sql("select 'src',src.{0} from md5_differences M inner join df_src_hashed src on M.Md5Hash=src.Md5Hash union all select 'tgt',tgt.{1} from md5_differences M1 inner join df_tgt_hashed tgt on M1.Md5Hash=tgt.Md5Hash order by {2}".format(source_column_names_str,target_column_names_str,source_column_names_str))
            md5_values_target.show()
            status=1
            return "Data discrepancies in " + str(round(diff_cnt)) + " records"
    
    except Exception as e:
        print("Failed With Exception : " + str(e))
        status=1
        
        
error=dataCompare(DF_SRC,DF_TGT) 

##### Save Result into AUDIT_TABLE 

QUERY_ID=spark.sql("select coalesce(max(QUERY_ID),0)+1 as QUERY_ID from AUDIT_TAB").collect()[0][0]
END_TIME = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
Insert_query = "insert into AUDIT_TAB values("+ str(QUERY_ID) + "," + str(TABLE_ID) + ",'" + START_TIME + "','" + END_TIME + "','" + error + "')" 
print(Insert_query)
spark.sql(Insert_query)
spark.sql("select * from AUDIT_TAB where Query_id=" + str(QUERY_ID) ).display()

Source Column are : ACTIVITY_KEY,ACTIVITY_CODE,ACTIVITY_DATE,ACTIVITY_DESCRIPTION,ACTIVITY_LONG_DESCRIPTION,ACTIVITY_TYPE_KEY,BILLING_ARRANGEMENT_KEY,CONTACT_KEY,CUSTOMER_KEY,EMPLOYEE_KEY,FINANCIAL_ACCOUNT_KEY,MIGRATION_IND,SERVICE_REQUEST_CASE_KEY,GS_INSERT_TS
Target Columns are : ACTIVITY_KEY,ACTIVITY_CODE,ACTIVITY_DATE,ACTIVITY_DESCRIPTION,ACTIVITY_LONG_DESCRIPTION,ACTIVITY_TYPE_KEY,BILLING_ARRANGEMENT_KEY,CONTACT_KEY,CUSTOMER_KEY,EMPLOYEE_KEY,FINANCIAL_ACCOUNT_KEY,MIGRATION_IND,SERVICE_REQUEST_CASE_KEY,GG_INSERT_TS
Source Column Count : 14
Target Column Count : 14
Column Count validation is successful
---------------------------------------------------------
Source Record Count : 107782
Target Record Count : 107782


Count validation is successful
---------------------------------------------------------
Data discrepancies are identified between the Source and Target. Total count of discrepancies is : 5941
['ACTIVITY_KEY', 'ACTIVITY_CODE', 'ACTIVITY_DATE', 'ACTIVITY_DESCRIPTION', 'ACTIVITY_LONG_DESCRIPTION', 'ACTIVITY_TYPE_KEY', 'BILLING_ARRANGEMENT_KEY', 'CONTACT_KEY', 'CUSTOMER_KEY', 'EMPLOYEE_KEY', 'FINANCIAL_ACCOUNT_KEY', 'MIGRATION_IND', 'SERVICE_REQUEST_CASE_KEY', 'GS_INSERT_TS']
['ACTIVITY_KEY', 'ACTIVITY_CODE', 'ACTIVITY_DATE', 'ACTIVITY_DESCRIPTION', 'ACTIVITY_LONG_DESCRIPTION', 'ACTIVITY_TYPE_KEY', 'BILLING_ARRANGEMENT_KEY', 'CONTACT_KEY', 'CUSTOMER_KEY', 'EMPLOYEE_KEY', 'FINANCIAL_ACCOUNT_KEY', 'MIGRATION_IND', 'SERVICE_REQUEST_CASE_KEY', 'GG_INSERT_TS']
+---+------------+-------------+-------------------+--------------------+-------------------------+-----------------+-----------------------+-----------+------------+------------+---------------------+-------------+------------------------+------------+
src|ACTIVITY_KEY|ACTIVITY_CODE| ACTIVITY_DATE|ACTIVITY_DESCRIPTION|ACTIVITY_LONG_DESCRIPTION|ACTIVITY_TYPE_KEY|BILLING_ARRANGEMENT_KEY|CONTACT_KEY|CUSTOMER_KEY|EMPLOYEE_KEY|FINANCIAL_ACCOUNT_KEY|MIGRATION_IND|SERVICE_REQUEST_CASE_KEY|GS_INSERT_TS|
+---+------------+-------------+-------------------+--------------------+-------------------------+-----------------+-----------------------+-----------+------------+------------+---------------------+-------------+------------------------+------------+
src| 11841| 90063|2019-08-30 17:36:12| Add ANM Subscriber| null| 10059| null| null| null| 10001| null| null| null| 2022-07-18|
tgt| 11841| 90063|2019-08-30 17:36:12| Add ANM Subscriber| null| 10059| null| 1206| null| 10001| null| null| null| 2022-07-18|
src| 11856| 90063|2019-08-30 17:43:41| Add ANM Subscriber| null| 10059| null| null| null| 10001| null| null| null| 2022-07-18|
tgt| 11856| 90063|2019-08-30 17:43:41| Add ANM Subscriber| null| 10059| null| 1207| null| 10001| null| null| null| 2022-07-18|
src| 11867| 90063|2019-08-30 17:44:45| Add ANM Subscriber| null| 10059| null| null| null| 10001| null| null| null| 2022-07-18|
tgt| 11867| 90063|2019-08-30 17:44:45| Add ANM Subscriber| null| 10059| null| 1208| null| 10001| null| null| null| 2022-07-18|
src| 11878| 90063|2019-08-30 17:51:07| Add ANM Subscriber| null| 10059| null| null| null| 10001| null| null| null| 2022-07-18|
tgt| 11878| 90063|2019-08-30 17:51:07| Add ANM Subscriber| null| 10059| null| 1209| null| 10001| null| null| null| 2022-07-18|
src| 11889| 90063|2019-08-30 17:52:17| Add ANM Subscriber| null| 10059| null| null| null| 10001| null| null| null| 2022-07-18|
tgt| 11889| 90063|2019-08-30 17:52:17| Add ANM Subscriber| null| 10059| null| 1210| null| 10001| null| null| null| 2022-07-18|
src| 11902| 90063|2019-08-30 17:57:02| Add ANM Subscriber| null| 10059| null| null| null| 10001| null| null| null| 2022-07-18|
tgt| 11902| 90063|2019-08-30 17:57:02| Add ANM Subscriber| null| 10059| null| 1211| null| 10001| null| null| null| 2022-07-18|
src| 11917| 90063|2019-08-30 17:58:12| Add ANM Subscriber| null| 10059| null| null| null| 10001| null| null| null| 2022-07-18|
tgt| 11917| 90063|2019-08-30 17:58:12| Add ANM Subscriber| null| 10059| null| 1212| null| 10001| nu

QUERY_ID,TABLE_ID,START_TIME,END_TIME,RESULT
2,1,2022-09-15 18:05:40,2022-09-15 18:05:55,Data discrepancies in 5941 records
